In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import IntegrityError
from psycopg2 import errors

In [2]:
twitter_article = pd.read_csv('merged_sentiment.csv')
result = pd.read_csv('house_election_d_r_3years.csv')

## Clean Result table

In [10]:
dem_2016 = result[(result['year'] == 2016) & (result['party'] == 'DEM')]
dem_2018 = result[(result['year'] == 2018) & (result['party'] == 'DEM')]
dem_2020 = result[(result['year'] == 2020) & (result['party'] == 'DEM')]

rep_2016 = result[(result['year'] == 2016) & (result['party'] == 'REP')]
rep_2018 = result[(result['year'] == 2018) & (result['party'] == 'REP')]
rep_2020 = result[(result['year'] == 2020) & (result['party'] == 'REP')]

In [27]:
result_dic = {'Year': [], 'Party': [], 'Norm_vote_count': []}
def combine_result(df):
    result_dic['Year'].append(df.iloc[0].loc['year'])
    result_dic['Party'].append(df.iloc[0].loc['party'])
    result_dic['Norm_vote_count'].append(sum(df['norm_vote_count']))

In [11]:
dem_2016

,year,region,party,norm_vote_count
0,2016,Midwest,DEM,3191.989391
2,2016,Northeast,DEM,4355.566460
4,2016,Southeast,DEM,4545.266460
6,2016,Southwest,DEM,1239.582164
8,2016,West,DEM,4266.983465


In [28]:
result_list = [dem_2016, dem_2018, dem_2020, rep_2016, rep_2018, rep_2020]
for item in result_list:
    combine_result(item)
print(result_dic)

{'Year': [2016, 2018, 2020, 2016, 2018, 2020], 'Party': ['DEM', 'DEM', 'DEM', 'REP', 'REP', 'REP'], 'Norm_vote_count': [17599.387940066972, 17720.805641928673, 20153.34372849364, 20059.20250137347, 17125.063021311315, 22590.162599503037]}


In [12]:
twitter_article

,Year,tw_Immigration,tw_Racial,tw_Abortion,tw_Socioeconomic,art_Immigration,art_Racial,art_Abortion,art_Socioeconomic
0,2016,-0.050966,-0.287514,-0.175296,0.078083,0.340021,0.295540,0.453646,0.564608
1,2017,-0.056006,-0.291790,-0.180437,0.078166,0.314661,0.366057,0.617787,0.579555
2,2018,-0.054213,-0.291755,-0.171763,0.078139,0.340941,0.415915,0.552726,0.660707
3,2019,-0.055452,-0.290693,-0.174275,0.078169,0.318504,0.382781,0.533270,0.695451
4,2020,-0.054611,-0.288259,-0.168331,0.078126,0.285201,0.213615,0.494138,0.591085
5,2021,-0.055628,-0.292840,-0.182862,0.078177,NaN,NaN,NaN,NaN
6,2022,-0.056524,-0.292167,-0.146302,0.078139,NaN,NaN,NaN,NaN


In [29]:
result_df = pd.DataFrame(result_dic)
result_df

,Year,Party,Norm_vote_count
0,2016,DEM,17599.387940
1,2018,DEM,17720.805642
2,2020,DEM,20153.343728
3,2016,REP,20059.202501
4,2018,REP,17125.063021
5,2020,REP,22590.162600


## Merging sentiment with vote count

In [30]:
# Assuming df1 and df2 are your DataFrames
final_table = pd.merge(twitter_article, result_df, on='Year', how='outer')
final_table

,Year,tw_Immigration,tw_Racial,tw_Abortion,tw_Socioeconomic,art_Immigration,art_Racial,art_Abortion,art_Socioeconomic,Party,Norm_vote_count
0,2016,-0.050966,-0.287514,-0.175296,0.078083,0.340021,0.295540,0.453646,0.564608,DEM,17599.387940
1,2016,-0.050966,-0.287514,-0.175296,0.078083,0.340021,0.295540,0.453646,0.564608,REP,20059.202501
2,2017,-0.056006,-0.291790,-0.180437,0.078166,0.314661,0.366057,0.617787,0.579555,NaN,NaN
3,2018,-0.054213,-0.291755,-0.171763,0.078139,0.340941,0.415915,0.552726,0.660707,DEM,17720.805642
4,2018,-0.054213,-0.291755,-0.171763,0.078139,0.340941,0.415915,0.552726,0.660707,REP,17125.063021
5,2019,-0.055452,-0.290693,-0.174275,0.078169,0.318504,0.382781,0.533270,0.695451,NaN,NaN
6,2020,-0.054611,-0.288259,-0.168331,0.078126,0.285201,0.213615,0.494138,0.591085,DEM,20153.343728
7,2020,-0.054611,-0.288259,-0.168331,0.078126,0.285201,0.213615,0.494138,0.591085,REP,22590.162600
8,2021,-0.055628,-0.292840,-0.182862,0.078177,NaN,NaN,NaN,NaN,NaN,NaN
9,2022,-0.056524,-0.292167,-0.146302,0.078139,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
final_table.to_csv('final_merge.csv', index=False)

In [33]:
copy_final = final_table.dropna()
copy_final

,Year,tw_Immigration,tw_Racial,tw_Abortion,tw_Socioeconomic,art_Immigration,art_Racial,art_Abortion,art_Socioeconomic,Party,Norm_vote_count
0,2016,-0.050966,-0.287514,-0.175296,0.078083,0.340021,0.295540,0.453646,0.564608,DEM,17599.387940
1,2016,-0.050966,-0.287514,-0.175296,0.078083,0.340021,0.295540,0.453646,0.564608,REP,20059.202501
3,2018,-0.054213,-0.291755,-0.171763,0.078139,0.340941,0.415915,0.552726,0.660707,DEM,17720.805642
4,2018,-0.054213,-0.291755,-0.171763,0.078139,0.340941,0.415915,0.552726,0.660707,REP,17125.063021
6,2020,-0.054611,-0.288259,-0.168331,0.078126,0.285201,0.213615,0.494138,0.591085,DEM,20153.343728
7,2020,-0.054611,-0.288259,-0.168331,0.078126,0.285201,0.213615,0.494138,0.591085,REP,22590.162600


## Model with all bias terms

In [43]:
#Multiple Linear Regression
import statsmodels.api as sm

# Prepare your features (sentiment scores) and target variable (election vote counts)
X = copy_final.iloc[:, 0:9]
y = copy_final.iloc[:, -1]

# Add a constant column to the feature data
X = sm.add_constant(X)

# Fit the multiple linear regression model
model = sm.OLS(y, X)
results = model.fit()

# Print the summary of the model fit
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:        Norm_vote_count   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sat, 20 May 2023   Prob (F-statistic):              0.147
Time:                        15:37:48   Log-Likelihood:                -50.045
No. Observations:                   6   AIC:                             106.1
Df Residuals:                       3   BIC:                             105.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              -374.8153    296.00

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## Model with only Immigration

In [54]:
# Prepare your features (sentiment scores) and target variable (election vote counts)
X_imm = copy_final[['Year', 'tw_Immigration', 'art_Immigration']]
y = copy_final.iloc[:, -1]

# Add a constant column to the feature data
X_imm = sm.add_constant(X_imm)

# Fit the multiple linear regression model
model_im = sm.OLS(y, X_imm)
results_im = model_im.fit()

# Print the summary of the model fit
print(results_im.summary())

                            OLS Regression Results                            
Dep. Variable:        Norm_vote_count   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sat, 20 May 2023   Prob (F-statistic):              0.147
Time:                        15:59:52   Log-Likelihood:                -50.045
No. Observations:                   6   AIC:                             106.1
Df Residuals:                       3   BIC:                             105.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            1.223e+05   1.27e+05     

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## Model with only Racial

In [55]:
# Prepare your features (sentiment scores) and target variable (election vote counts)
X_ra = copy_final[['Year', 'tw_Racial', 'art_Racial']]
y = copy_final.iloc[:, -1]

# Add a constant column to the feature data
X_ra = sm.add_constant(X_ra)

# Fit the multiple linear regression model
model_ra = sm.OLS(y, X_ra)
results_ra = model_ra.fit()

# Print the summary of the model fit
print(results_ra.summary())

                            OLS Regression Results                            
Dep. Variable:        Norm_vote_count   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sat, 20 May 2023   Prob (F-statistic):              0.147
Time:                        16:01:07   Log-Likelihood:                -50.045
No. Observations:                   6   AIC:                             106.1
Df Residuals:                       3   BIC:                             105.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.631e+05   3.63e+05     -0.724      0.5

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## Model with only Abortion

In [57]:
# Prepare your features (sentiment scores) and target variable (election vote counts)
X_ab = copy_final[['Year', 'tw_Abortion', 'art_Abortion']]
y = copy_final.iloc[:, -1]

# Add a constant column to the feature data
X_ab = sm.add_constant(X_ab)

# Fit the multiple linear regression model
model_ab = sm.OLS(y, X_ab)
results_ab = model_ab.fit()

# Print the summary of the model fit
print(results_ab.summary())

                            OLS Regression Results                            
Dep. Variable:        Norm_vote_count   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sat, 20 May 2023   Prob (F-statistic):              0.147
Time:                        16:02:56   Log-Likelihood:                -50.045
No. Observations:                   6   AIC:                             106.1
Df Residuals:                       3   BIC:                             105.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.339e+05   5.44e+04     -2.460   

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## Model with only Socioeconomic

In [58]:
# Prepare your features (sentiment scores) and target variable (election vote counts)
X_so = copy_final[['Year', 'tw_Socioeconomic', 'art_Socioeconomic']]
y = copy_final.iloc[:, -1]

# Add a constant column to the feature data
X_so = sm.add_constant(X_so)

# Fit the multiple linear regression model
model_so = sm.OLS(y, X_so)
results_so = model_so.fit()

# Print the summary of the model fit
print(results_so.summary())

                            OLS Regression Results                            
Dep. Variable:        Norm_vote_count   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sat, 20 May 2023   Prob (F-statistic):              0.147
Time:                        16:04:09   Log-Likelihood:                -50.045
No. Observations:                   6   AIC:                             106.1
Df Residuals:                       3   BIC:                             105.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1.669e+06   7.46e+0

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## Model with only twitter

In [59]:
# Prepare your features (sentiment scores) and target variable (election vote counts)
X_tw = copy_final[['Year', 'tw_Immigration', 'tw_Racial', 'tw_Abortion', 'tw_Socioeconomic']]
y = copy_final.iloc[:, -1]

# Add a constant column to the feature data
X_tw = sm.add_constant(X_tw)

# Fit the multiple linear regression model
model_tw = sm.OLS(y, X_tw)
results_tw = model_tw.fit()

# Print the summary of the model fit
print(results_tw.summary())

                            OLS Regression Results                            
Dep. Variable:        Norm_vote_count   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sat, 20 May 2023   Prob (F-statistic):              0.147
Time:                        16:05:57   Log-Likelihood:                -50.045
No. Observations:                   6   AIC:                             106.1
Df Residuals:                       3   BIC:                             105.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.013e+05    4.4e+04  

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


## Model with Article only

In [60]:
# Prepare your features (sentiment scores) and target variable (election vote counts)
X_ar = copy_final[['Year', 'art_Immigration', 'art_Racial', 'art_Abortion', 'art_Socioeconomic']]
y = copy_final.iloc[:, -1]

# Add a constant column to the feature data
X_ar = sm.add_constant(X_ar)

# Fit the multiple linear regression model
model_ar = sm.OLS(y, X_ar)
results_ar = model_ar.fit()

# Print the summary of the model fit
print(results_ar.summary())

                            OLS Regression Results                            
Dep. Variable:        Norm_vote_count   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     3.894
Date:                Sat, 20 May 2023   Prob (F-statistic):              0.147
Time:                        16:07:52   Log-Likelihood:                -50.045
No. Observations:                   6   AIC:                             106.1
Df Residuals:                       3   BIC:                             105.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              -377.3649    298.14

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
